In [3]:
from distribution_fit_class import DistributionFit
from portfolio_optimization_class import PortfolioOptimization, optimize_windows
import pandas as pd
import numpy as np


In [4]:
distribution_fit = DistributionFit()
file_path = r'data\random_data\n_stocks_per_sector.csv'
distribution_fit.load_df_from_csv(file_path)

returns_df = pd.read_csv(r'data\random_data\n_stocks_per_sector.csv', index_col=0, parse_dates=True)

copulas = ["clayton_random", "gaussian", "t_student"]
distributions = ["gauss_dist", "t_dist"]
window_size = 32

In [7]:
taus = [0.0005, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.15, 0.2]
maximum_weight = 0.3


In [ ]:
for tau in taus:
    portfolios = optimize_windows(number_of_quarters=window_size, tau=tau, maximum_weight=maximum_weight, data_path=file_path)
    for copula in copulas:
        for window in range(0, 64):
            if window % 5 == 0:
                print(f'Iter: {copula} - {window}, tau: {tau}')
            returns_window = returns_df.iloc[window : 32+window]
            distribution_fit = DistributionFit()
            distribution_fit.set_df(returns_window)
            distribution_fit.fit_distribution_all_stocks()
            # distribution_fit.get_fitted_params()
            simulated_data_path = r'../copulas_outputs/simulated_' + copula + '_22_stocks' + f'_{window}_window.csv'
            distribution_fit.load_simulated_data_from_csv(simulated_data_path)
            distribution_fit.calculate_returns_from_simulated_quantiles()
            returns_norm = distribution_fit.get_simulated_return_norm()
            returns_t = distribution_fit.get_simulated_return_t_student()
            returns_list = [(returns_norm, "gauss_dist"), (returns_t, "t_dist")]
            # output_file_path = r'../data_return_rates/simulated_' + copula + '_RETURNS_22_stocks' + f'_{window}_window_'
            # returns_norm.to_csv(output_file_path + "gauss_dist.csv")
            # returns_t.to_csv(output_file_path + "t_dist.csv")
            for simulated_returns, dist in returns_list:
                weights = portfolios.iloc[window].filter(regex="^w\d+$").values
                expectile = -portfolios.iloc[window]["EVAR"]
                weighted_returns = simulated_returns.values * weights

                validation_values = (1 - tau) * np.minimum(weighted_returns - expectile, 0) - tau * np.maximum(weighted_returns - expectile, 0)
                scoring_values = (1 - tau) * np.minimum((weighted_returns - expectile)**2, 0) + tau * np.maximum((weighted_returns - expectile)**2, 0)

                expected_validation_values = np.mean(validation_values, axis=1)
                expected_scoring_values = np.mean(scoring_values, axis=1)

                validation_df = pd.DataFrame({
                    'Portfolio Return': np.sum(weighted_returns, axis=1),
                    'Expected Validation Value': expected_validation_values,
                    'Expected Scoring Value': expected_scoring_values
                })
                tau_str = str(tau).replace(".", "_")
                output_path = r"../scores/taus/" + f'tau_{tau_str}/{copula}_22_stocks_{window}_window_{dist}_tau_{tau_str}_SCORES.csv'
                validation_df.to_csv(output_path)


Iter: clayton_random - 0, tau: 0.0005
Iter: clayton_random - 5, tau: 0.0005
Iter: clayton_random - 10, tau: 0.0005
Iter: clayton_random - 15, tau: 0.0005
Iter: clayton_random - 20, tau: 0.0005
Iter: clayton_random - 25, tau: 0.0005
Iter: clayton_random - 30, tau: 0.0005
Iter: clayton_random - 35, tau: 0.0005
Iter: clayton_random - 40, tau: 0.0005
Iter: clayton_random - 45, tau: 0.0005
Iter: clayton_random - 50, tau: 0.0005
Iter: clayton_random - 55, tau: 0.0005
Iter: clayton_random - 60, tau: 0.0005


c:\Users\Miłosz\.conda\envs\cplex_env\lib\site-packages\scipy\stats\_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)
c:\Users\Miłosz\.conda\envs\cplex_env\lib\site-packages\scipy\stats\_distn_infrastructure.py:2023: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
c:\Users\Miłosz\.conda\envs\cplex_env\lib\site-packages\scipy\stats\_distn_infrastructure.py:2024: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
c:\Users\Miłosz\.conda\envs\cplex_env\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in minimum
c:\Users\Miłosz\.conda\envs\cplex_env\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in maximum
c:\Users\Miłosz\.conda\envs\cplex_env\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in minimum
c:\Users\Miłosz\.c

Iter: gaussian - 0, tau: 0.0005
Iter: gaussian - 5, tau: 0.0005
Iter: gaussian - 10, tau: 0.0005
Iter: gaussian - 15, tau: 0.0005
Iter: gaussian - 20, tau: 0.0005
Iter: gaussian - 25, tau: 0.0005
Iter: gaussian - 30, tau: 0.0005
Iter: gaussian - 35, tau: 0.0005
Iter: gaussian - 40, tau: 0.0005
Iter: gaussian - 45, tau: 0.0005
Iter: gaussian - 50, tau: 0.0005
Iter: gaussian - 55, tau: 0.0005
Iter: gaussian - 60, tau: 0.0005
Iter: t_student - 0, tau: 0.0005
Iter: t_student - 5, tau: 0.0005
Iter: t_student - 10, tau: 0.0005
Iter: t_student - 15, tau: 0.0005
Iter: t_student - 20, tau: 0.0005
Iter: t_student - 25, tau: 0.0005
Iter: t_student - 30, tau: 0.0005
Iter: t_student - 35, tau: 0.0005
Iter: t_student - 40, tau: 0.0005
Iter: t_student - 45, tau: 0.0005
Iter: t_student - 50, tau: 0.0005
Iter: t_student - 55, tau: 0.0005
Iter: t_student - 60, tau: 0.0005
Iter: clayton_random - 0, tau: 0.001
Iter: clayton_random - 5, tau: 0.001
Iter: clayton_random - 10, tau: 0.001
Iter: clayton_random 